In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('/content/diabetes.csv')

In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
df.corr()['Outcome'].sort_values(ascending=False)

,Outcome
Outcome,1.000000
Glucose,0.466581
BMI,0.292695
Age,0.238356
Pregnancies,0.221898
DiabetesPedigreeFunction,0.173844
Insulin,0.130548
SkinThickness,0.074752
BloodPressure,0.065068


In [ ]:
df.shape

(768, 9)

In [ ]:
X=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.fit(X_train_scaled,y_train,batch_size=32, epochs=10,validation_data=(X_test_scaled,y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.6410 - loss: 0.6238 - val_accuracy: 0.6948 - val_loss: 0.6033
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7216 - loss: 0.5638 - val_accuracy: 0.6948 - val_loss: 0.5752
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7400 - loss: 0.5273 - val_accuracy: 0.7013 - val_loss: 0.5539
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7359 - loss: 0.5173 - val_accuracy: 0.7078 - val_loss: 0.5398
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7512 - loss: 0.4995 - val_accuracy: 0.7338 - val_loss: 0.5288
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7490 - loss: 0.4965 - val_accuracy: 0.7273 - val_loss: 0.5230
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7786 - loss: 0.4795 - val_accuracy: 0.7403 - val_loss: 0.5158
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7834 - loss: 0.4661 - val_accuracy: 0.7468 - val_loss

In [ ]:
pip install -U keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 5.0 MB/s eta 0:00:00


In [ ]:
import kerastuner as kt

<ipython-input-15-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
def build_model (hp):

  model=Sequential()
  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [ ]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [ ]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7077922224998474

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 19s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train_scaled,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test_scaled,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.7296 - loss: 0.5395 - val_accuracy: 0.7468 - val_loss: 0.5241
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7308 - loss: 0.5486 - val_accuracy: 0.7597 - val_loss: 0.5160
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7583 - loss: 0.5022 - val_accuracy: 0.7597 - val_loss: 0.5100
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7630 - loss: 0.4965 - val_accuracy: 0.7662 - val_loss: 0.5049
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7593 - loss: 0.4964 - val_accuracy: 0.7662 - val_loss: 0.5031
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7535 - loss: 0.4902 - val_accuracy: 0.7662 - val_loss: 0.4998
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7694 - loss: 0.4646 - val_accuracy: 0.7532 - val_loss: 0.4978
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7810 - loss: 0.4524 - val_accuracy

In [ ]:
def build_model(hp):

  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective = 'val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='Keras_tuner'
                        )

In [ ]:
tuner.search(X_train_scaled,y_train, epochs=5, validation_data= (X_test_scaled,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 12s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train_scaled,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data= (X_test_scaled,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7872 - loss: 0.4070 - val_accuracy: 0.7403 - val_loss: 0.5577
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8323 - loss: 0.3678 - val_accuracy: 0.7468 - val_loss: 0.5571
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8115 - loss: 0.3741 - val_accuracy: 0.7403 - val_loss: 0.5567
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8519 - loss: 0.3521 - val_accuracy: 0.7468 - val_loss: 0.5556
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8239 - loss: 0.3744 - val_accuracy: 0.7338 - val_loss: 0.5573
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8126 - loss: 0.3784 - val_accuracy: 0.7273 - val_loss: 0.5561
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7894 - loss: 0.4135 - val_accuracy: 0.7338 - val_loss: 0.5569
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8175 - loss: 0.3676 - val_accuracy: 0.727

In [ ]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [ ]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='Keras_tuner_1')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 19s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 7}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.fit(X_train_scaled,y_train,epochs=100,initial_epoch=6,validation_data=(X_test_scaled,y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 7/100


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.7815 - loss: 0.5114 - val_accuracy: 0.7727 - val_loss: 0.5790
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7707 - loss: 0.4815 - val_accuracy: 0.7273 - val_loss: 0.5352
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7952 - loss: 0.4511 - val_accuracy: 0.8117 - val_loss: 0.5688
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7879 - loss: 0.4417 - val_accuracy: 0.7532 - val_loss: 0.5679
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8063 - loss: 0.4278 - val_accuracy: 0.7597 - val_loss: 0.5676
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7967 - loss: 0.4152 - val_accuracy: 0.6948 - val_loss: 0.5880
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8063 - loss: 0.4128 - val_accuracy: 0.7597 - val_loss: 0.6060
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8062 - loss: 0.4277 - val_accuracy: 0.7727 - val_los

In [ ]:
def build_model(hp):

  model=Sequential()


  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:

      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid']),
              input_dim = 8

          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:

      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value=8, max_value=128, step=8),
              activation = hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid'])
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1

    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

  return model


In [ ]:
tuner= kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=3,
                       directory='mydir',
                       project_name='final2')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
tuner.search(X_train_scaled,y_train,epochs=5,validation_data=(X_test_scaled,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 14s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 112,
 'activation0': 'sigmoid',
 'dropout0': 0.7,
 'optimizer': 'sgd',
 'units1': 8,
 'activation1': 'relu',
 'dropout1': 0.1,
 'units2': 8,
 'activation2': 'relu',
 'dropout2': 0.1,
 'units3': 8,
 'activation3': 'relu',
 'dropout3': 0.1,
 'units4': 8,
 'activation4': 'relu',
 'dropout4': 0.1,
 'units5': 8,
 'activation5': 'relu',
 'dropout5': 0.1}

In [ ]:
model=tuner.get_best_models(num_models=1)[0]
model.fit(X_train_scaled,y_train,epochs=200,validation_data=(X_test_scaled,y_test))

Epoch 1/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.6018 - loss: 0.6889 - val_accuracy: 0.6429 - val_loss: 0.6818
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6335 - loss: 0.6813 - val_accuracy: 0.6429 - val_loss: 0.6760
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6234 - loss: 0.6783 - val_accuracy: 0.6429 - val_loss: 0.6714
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6454 - loss: 0.6701 - val_accuracy: 0.6429 - val_loss: 0.6676
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6739 - loss: 0.6601 - val_accuracy: 0.6429 - val_loss: 0.6645
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6713 - loss: 0.6559 - val_accuracy: 0.6429 - val_loss: 0.6623
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6304 - loss: 0.6654 - val_accuracy: 0.6429 - val_loss: 0.6607
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6763 - loss: 0.6505 - val_accuracy: 0.6429 - 